<a href="https://colab.research.google.com/github/hoangtheanhhp/ZaloQA/blob/bert_lstm/QASystem/run_zalo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!rm -rf zalo
!test -d zalo || git clone https://github.com/hoangtheanhhp/ZaloQA zalo
import sys
if 'zalo' not in sys.path:
  sys.path+=['zalo']

Cloning into 'zalo'...
remote: Enumerating objects: 185, done.
remote: Counting objects: 100% (185/185), done.
remote: Compressing objects: 100% (146/146), done.
remote: Total 701 (delta 115), reused 87 (delta 38), pack-reused 516
Receiving objects: 100% (701/701), 95.38 MiB | 26.55 MiB/s, done.
Resolving deltas: 100% (343/343), done.
Checking out files: 100% (42/42), done.


In [3]:
%tensorflow_version 1.x
!pip install kashgari==1.1.4



TensorFlow 1.x selected.


In [0]:
DATASET_PATH = "zalo/Dataset/zalo"
SAVE_MODEL_PATH = "/content/drive/My Drive/ZaloAI/result/model_squad_data"
MODEL_PATH = "/content/drive/My Drive/ZaloAI/bert"

In [5]:
from zalo.QASystem.preprocess import ZaloDatasetProcessor

import kashgari
processor = ZaloDatasetProcessor()
from kashgari.embeddings import BERTEmbedding

bert_embedding = BERTEmbedding(MODEL_PATH,
                               task=kashgari.CLASSIFICATION,
                               sequence_length=256)

tokenizer = bert_embedding.tokenizer

/usr/local/lib/python3.6/dist-packages/bert4keras/__init__.py:22: UserWarning: bert4keras.bert has been renamed as bert4keras.models.
  warnings.warn('bert4keras.bert has been renamed as bert4keras.models.')
/usr/local/lib/python3.6/dist-packages/bert4keras/__init__.py:23: UserWarning: please use bert4keras.models.
  warnings.warn('please use bert4keras.models.')
/usr/local/lib/python3.6/dist-packages/bert4keras/__init__.py:44: UserWarning: bert4keras.tokenizer has been renamed as bert4keras.tokenizers.
  warnings.warn('bert4keras.tokenizer has been renamed as bert4keras.tokenizers.')
/usr/local/lib/python3.6/dist-packages/bert4keras/__init__.py:45: UserWarning: please use bert4keras.tokenizers.
  warnings.warn('please use bert4keras.tokenizers.')


In [0]:
# Our tokenizer already added the BOS([CLS]) and EOS([SEP]) token
# so we need to disable the default add_bos_eos setting.
bert_embedding.processor.add_bos_eos = False

In [0]:
processor.val_data = []
processor.train_data = []

In [0]:
processor.load_from_path("/content/zalo/Dataset","squad","squad_vi.json")

In [9]:
true_squad = []
false_squad = []
for data in processor.train_data:
  if data.get('label'):
    true_squad.append(data)
  else:
    false_squad.append(data)
print(len(true_squad))
print(len(false_squad))
processor.train_data = true_squad[:30000]
processor.train_data.extend(false_squad[0:22000])

49428
80104


In [10]:
processor.load_from_path(DATASET_PATH,"train", "train_full.json")
processor.load_from_path("/content/zalo/Dataset/zalo", "test","test_full.json")

data = processor.train_data
val = processor.test_data
train_x = []
train_y = []
val_x = []
val_y = []
for d in data:
  if d.get('text') and len(d.get('text')) > 0:
    train_x.append(tokenizer.tokenize(d.get('question')) + 
                    tokenizer.tokenize(d.get('text'))[1:])
    train_y.append(str(d.get('label', False)))

for d in val:
  val_x.append(tokenizer.tokenize(d.get('question')) + 
               tokenizer.tokenize(d.get('text'))[1:])
  val_y.append(str(d.get('label', False)))

print(len(train_x), len(train_y), len(val_x), len(val_y))


100%|██████████| 2061/2061 [00:00<00:00, 569839.19it/s]


69781 69781 2061 2061


In [0]:
from kashgari.tasks.classification import BiLSTM_Model

import logging
logging.basicConfig(level='DEBUG')

model = BiLSTM_Model(bert_embedding)

model.fit(train_x,
          train_y,
          val_x,
          val_y,
          epochs=3,
          batch_size=256)

model.save(SAVE_MODEL_PATH)


Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        [(None, 256)]        0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      [(None, 256)]        0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 256, 768), ( 91812096    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 256, 768)     1536        Input-Segment[0][0]              
____________________________________________________________________________________________

In [0]:
from kashgari.utils import load_model

model = load_model(SAVE_MODEL_PATH)

In [0]:

test = processor.test_data
test_x = []
test_y = []
for d in test:
  test_x.append(tokenizer.tokenize(d.get('question')) + 
               tokenizer.tokenize(d.get('text'))[1:])
  test_y.append(str(d.get('label')))

In [0]:
# evaluate the model
model.evaluate(test_x, test_y)